In [13]:
import numpy as np
import os
import nibabel as nib

from scipy import ndimage
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [3]:
def parse_record(tfrecord):
    attributes = tf.io.parse_single_example(tfrecord, features={
        'x_shape': tf.io.FixedLenFeature([3], tf.int64),
        'x': tf.io.FixedLenFeature([], tf.string),
        'y': tf.io.FixedLenFeature([], tf.int64)
    })
    x_shape = attributes['x_shape']
    x = tf.io.decode_raw(attributes['x'], tf.float64)
    x = tf.reshape(x, x_shape)
    y = attributes['y']
    return x, y

In [9]:
train_data = tf.data.TFRecordDataset('train_data.tfrecord')

train_dataset = train_data.map(parse_record)
train_dataset = train_data.batch(2)
train_dataset = train_data.prefetch(buffer_size=tf.data.AUTOTUNE)

In [17]:
test = next(iter(train_dataset))

In [27]:
for ele in test:
    print(ele)

TypeError: Cannot iterate over a scalar tensor.

TypeError: 'PrefetchDataset' object is not subscriptable

In [14]:
def create_model(width=128, height=128, depth=64):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model

In [15]:

model = create_model()

initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr_schedule), metrics='accuracy')

batch_size = 1
epochs = 2
validation_split = 0.1

with tf.device("GPU"):
    for train, test in train_data:
        model.fit(
            train_dataset,
            epochs=epochs,
            shuffle=True,
            verbose=2
        )



TypeError: Cannot iterate over a scalar tensor.